### 2.1 Overfitting and Regularization 

You might be wondering, “Why would we ever want to throw variables out,
can’t that only hurt our model?”

The primary answer: it helps us avoid a common issue called **overfitting**.

Overfitting occurs when a model that specializes its coefficients too much on the
data it was trained on, and only to perform poorly when predicting on data
outside the training set.

The extreme example of overfitting is a model that can perfectly memorize the
training data, but can do no better than just randomly guess when predicting
on a new observation.

The techniques applied to reduce overfitting are known as **regularization**.

Regularization is an attempt to limit a model’s ability to specialize too narrowly
on training data (e.g. limit overfitting) by penalizing extreme values of the
model’s parameters.

The additional term in the lasso regression loss function ($ \alpha ||\beta||_1 $)
is a form of regularization.

Let’s demonstrate the overfitting and regularization phenomenon on our housing
price data as follows:

1. Split the data set into training and testing subsets. We will use the first 50 observations for training and the rest for testing.  
1. Fit the linear regression model and report MSE on training and testing datasets.  
1. Fit the lasso model and report the same statistics.  



### Dataset

Let’s load the dataset and take a quick look at our task.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

colors = ['#165aa7', '#cb495c', '#fec630', '#bb60d5', '#f47915', '#06ab54', '#002070', '#b27d12', '#007030']

# We will import all these here to ensure that they are loaded, but
# will usually re-import close to where they are used to make clear
# where the functions come from
from sklearn import (
    linear_model, metrics, neural_network, pipeline, model_selection
)

url = "https://datascience.quantecon.org/assets/data/kc_house_data.csv"
df = pd.read_csv(url)
df.info()

X = df.drop(["price", "date", "id"], axis=1).copy()
# convert everything to be a float for later on
for col in list(X):
    X[col] = X[col].astype(float)
X.head()
y = np.log(df["price"])
df["log_price"] = y
y.head()

## Lasso Regression

Lasso regression is very closely related to linear regression.

The lasso model also generates predictions using $ y = X \beta $ but
optimizes over a slightly different loss function.

The optimization problem solved by lasso regression can be written as

$$
\min_{\beta} {|| X \beta - y||_2}^2 + \underbrace{\alpha {|| \beta ||_1}}_{\text{new part}}
$$

where $ || a ||_1 = \sum_{i=1}^N | a_i| $ is the [l1-norm](http://mathworld.wolfram.com/L1-Norm.html) and $ \alpha $ is called the regularization parameter.

The additional term penalizes large coefficients and in practice, effectively sets coefficients to zero
for features that are not informative about the
target.

Let’s see an example of what this looks like using the full feature set in
`X`.

In [ ]:
lasso_model = linear_model.Lasso()
lasso_model.fit(X, y)

lasso_coefs = pd.Series(dict(zip(list(X), lasso_model.coef_)))
lr_coefs = pd.Series(dict(zip(list(X), lr_model.coef_)))
coefs = pd.DataFrame(dict(lasso=lasso_coefs, linreg=lr_coefs))
coefs

Notice that many coefficients from the lasso regression have been set to
zero.

The intuition here is that the corresponding features hadn’t provided
enough predictive power to be worth considering alongside the other features.

The default value for the $ \alpha $ parameter is 1.0.

Larger $ \alpha $ values cause coefficients to shrink (and maybe
additional ones to be thrown out).

In [ ]:
# Compute lasso for many alphas (the lasso path)
from itertools import cycle
alphas = np.exp(np.linspace(10, -2, 50))
alphas, coefs_lasso, _ = linear_model.lasso_path(X, y, alphas=alphas, max_iter=10000)

# plotting
fig, ax = plt.subplots(figsize=(12, 8))
color_cycle = cycle(colors)
log_alphas = -np.log10(alphas)
for coef_l, c, name in zip(coefs_lasso, color_cycle, list(X)):
   ax.plot(log_alphas, coef_l, c=c)
   ax.set_xlabel('-Log(alpha)')
   ax.set_ylabel('lasso coefficients')
   ax.set_title('Lasso Path')
   ax.axis('tight')
   maxabs = np.max(np.abs(coef_l))
   i = [idx for idx in range(len(coef_l)) if abs(coef_l[idx]) >= (0.9*maxabs)][0]
   xnote = log_alphas[i]
   ynote = coef_l[i]
   ax.annotate(name, (xnote, ynote), color=c)

In [ ]:
def fit_and_report_mses(mod, X_train, X_test, y_train, y_test):
    mod.fit(X_train, y_train)
    return dict(
        mse_train=metrics.mean_squared_error(y_train, mod.predict(X_train)),
        mse_test=metrics.mean_squared_error(y_test, mod.predict(X_test))
    )

n_test = 50
X_train = X.iloc[:n_test, :]
X_test = X.iloc[n_test:, :]
y_train = y.iloc[:n_test]
y_test = y.iloc[n_test:]

fit_and_report_mses(linear_model.LinearRegression(), X_train, X_test, y_train, y_test)

In [ ]:
fit_and_report_mses(linear_model.Lasso(), X_train, X_test, y_train, y_test)

Notice how the MSE on the training dataset was smaller for the linear model
without the regularization, but the MSE on the test dataset was much
higher.

This strongly suggests that the linear regression model was
overfitting.

The regularization parameter has a large impact on overfitting.

In [ ]:
alphas = np.exp(np.linspace(10, -5, 100))
mse = pd.DataFrame([fit_and_report_mses(linear_model.Lasso(alpha=alpha, max_iter=50000),
                           X_train, X_test, y_train, y_test)
                    for alpha in alphas])
mse["log_alpha"] = -np.log10(alphas)
fig, ax = plt.subplots(figsize=(10,6))
mse.plot(x="log_alpha", y="mse_test", c=colors[0], ax=ax)
mse.plot(x="log_alpha", y="mse_train", c=colors[1], ax=ax)
ax.set_xlabel(r"$-\log(\alpha)$")
ax.set_ylabel("MSE")
ax.get_legend().remove()
ax.annotate("test",(mse.log_alpha[15], mse.mse_test[15]),color=colors[0])
ax.annotate("train",(mse.log_alpha[30], mse.mse_train[30]),color=colors[1])

### Cross-validation of Regularization Parameter

As you can see in the figure above, the regularization parameter has a
large impact on MSE in the test data.

Moreover, the relationship between the test data MSE and $ \alpha $ is
complicated and non-monotonic.

One popular method for choosing the regularization parameter is cross-validation.

Roughly speaking, cross-validation splits the dataset into many training/testing
subsets, then chooses the regularization parameter value that minimizes the
average MSE.

More precisely, k-fold cross-validation does the following:

1. Partition the dataset randomly into k subsets/”folds”.  
1. Compute $ MSE_j(\alpha)= $ mean squared error in j-th subset
  when using the j-th subset as test data, and other k-1 as training
  data.  
1. Minimize average (across folds) MSE $ \min_\alpha \frac{1}{k}
  \sum_{j=1}^k MSE_j(\alpha) $.  


The following code plots 5-fold, cross-validated MSE as a function of
$ \alpha $, using the same training data as above.

In [ ]:
from sklearn.model_selection import cross_val_score
mse["cv"] = [-np.mean(cross_val_score(linear_model.Lasso(alpha=alpha, max_iter=50000),
                                  X_train, y_train, cv=5, scoring='neg_mean_squared_error'))
          for alpha in alphas]
mse.plot(x="log_alpha", y="cv", c=colors[2], ax=ax)
ax.annotate("5 fold cross-validation", (mse.log_alpha[40], mse.cv[40]), color=colors[2])
ax.get_legend().remove()
ax.set_xlabel(r"$-\log(\alpha)$")
ax.set_ylabel("MSE")
fig

scikit learn also includes methods to automate the above and select
$ \alpha $.

In [ ]:
# LassoCV exploits special structure of lasso problem to minimize CV more efficiently
lasso = linear_model.LassoCV(cv=5).fit(X_train,y_train)
-np.log10(lasso.alpha_) # should roughly = minimizer on graph, not exactly equal due to random splitting

### Holdout

Practitioners often use another technique to avoid overfitting, called
*holdout*.

We demonstrated an extreme example of applying holdout above when we used only
the first 50 observations to train our models.

In general, good practice is to split the entire dataset into a training subset
and testing or validation subset.

The splitting should be done randomly. It should leave enough data in the
training dataset to produce a good model, but also enough in the validation
subset to determine the degree of overfitting.

There aren’t hard and fast rules for how much data to put in each subset, but a
reasonable default uses 75% of the data for training and the
rest for testing.

As in the example above, the training data is often further split
while selecting regularization parameters with cross-validation.

The `sklearn` function `model_selection.train_test_split` will do this for you:

In [ ]:
# note test_size=0.25 is the default value, but is shown here so you
# can see how to change it
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.25)

### Exercise 5

Experiment with how the size of the holdout dataset can impact a diagnosis
of overfitting.

Evaluate only the `LinearRegression` model on the full feature set and use
the `model_selection.train_test_split` function with various values for
`test_size`.

### Lasso in Econometrics

Lasso is becoming increasingly popular in economics, partially due to
the work of Victor Chernozhukov and his coauthors.

In econometrics, the goal is typically to estimate some coefficient of interest or causal
effect, rather than obtaining the most precise prediction.

Among other things, this different goal affects how the regularization parameter
must be chosen.

[[regBC11](#id23)] and [[regCHS16](#id21)] are somewhat approachable introductions
to this area.

The latter of these two references includes an R package.

[[regCCD+18](#id14)] reflects close to the state-of-art in this rapidly
advancing area.

## References

Two good text books covering the above regression methods are
[[regFHT09](#id26)] and [[regEH16](#id27)]

<a id='id24'></a>
\[regAI18\] Susan Athey and Guido Imbens. Machine learning and econometrics. 2018. URL: [https://www.aeaweb.org/conference/cont-ed/2018-webcasts](https://www.aeaweb.org/conference/cont-ed/2018-webcasts).

<a id='id25'></a>
\[regAI17\] Susan Athey and Guido W. Imbens. The state of applied econometrics: causality and policy evaluation. *Journal of Economic Perspectives*, 31(2):3–32, May 2017. URL: [http://www.aeaweb.org/articles?id=10.1257/jep.31.2.3](http://www.aeaweb.org/articles?id=10.1257/jep.31.2.3), [doi:10.1257/jep.31.2.3](https://doi.org/10.1257/jep.31.2.3).

<a id='id23'></a>
\[regBC11\] Alexandre Belloni and Victor Chernozhukov. *High Dimensional Sparse Econometric Models: An Introduction*, pages 121–156. Springer Berlin Heidelberg, Berlin, Heidelberg, 2011. URL: [https://doi.org/10.1007/978-3-642-19989-9_3](https://doi.org/10.1007/978-3-642-19989-9_3), [doi:10.1007/978-3-642-19989-9_3](https://doi.org/10.1007/978-3-642-19989-9_3).

<a id='id14'></a>
\[regCCD+18\] Victor Chernozhukov, Denis Chetverikov, Mert Demirer, Esther Duflo, Christian Hansen, Whitney Newey, and James Robins. Double/debiased machine learning for treatment and structural parameters. *The Econometrics Journal*, 21(1):C1–C68, 2018. URL: [https://onlinelibrary.wiley.com/doi/abs/10.1111/ectj.12097](https://onlinelibrary.wiley.com/doi/abs/10.1111/ectj.12097), [arXiv:https://onlinelibrary.wiley.com/doi/pdf/10.1111/ectj.12097](https://arxiv.org/abs/https://onlinelibrary.wiley.com/doi/pdf/10.1111/ectj.12097), [doi:10.1111/ectj.12097](https://doi.org/10.1111/ectj.12097).

<a id='id21'></a>
\[regCHS16\] Victor Chernozhukov, Chris Hansen, and Martin Spindler. hdm: high-dimensional metrics. *R Journal*, 8(2):185–199, 2016. URL: [https://journal.r-project.org/archive/2016/RJ-2016-040/index.html](https://journal.r-project.org/archive/2016/RJ-2016-040/index.html).

<a id='id27'></a>
\[regEH16\] Bradley Efron and Trevor Hastie. *Computer age statistical inference*. Volume 5. Cambridge University Press, 2016. URL: [https://web.stanford.edu/~hastie/CASI/](https://web.stanford.edu/~hastie/CASI/).

<a id='id26'></a>
\[regFHT09\] Jerome Friedman, Trevor Hastie, and Robert Tibshirani. *The elements of statistical learning*. Springer series in statistics, 2009. URL: [https://web.stanford.edu/~hastie/ElemStatLearn/](https://web.stanford.edu/~hastie/ElemStatLearn/).

<a id='id28'></a>
\[regHSW89\] Kurt Hornik, Maxwell Stinchcombe, and Halbert White. Multilayer feedforward networks are universal approximators. *Neural Networks*, 2(5):359 – 366, 1989. URL: [http://www.sciencedirect.com/science/article/pii/0893608089900208](http://www.sciencedirect.com/science/article/pii/0893608089900208), [doi:https://doi.org/10.1016/0893-6080(89)90020-8](https://doi.org/https://doi.org/10.1016/0893-6080%2889%2990020-8).


<a id='app-reg-ex'></a>

## Exercises

### Exercise 1

Use the `sqft_lr_model` that we fit to generate predictions for all data points
in our sample.

Note that you need to pass a DataFrame (not Series)
containing the `sqft_living` column to the `predict`. (See how we passed that to `.fit`
above for help)

Make a scatter chart with the actual data and the predictions on the same
figure. Does it look familiar?

When making the scatter for model predictions, we recommend passing
`c="red"` and `alpha=0.25` so you can distinguish the data from
predictions.

In [ ]:
# Generate predictions

# Plot
fig, ax = plt.subplots()

# Make scatter of data

# Make scatter of predictions

([back to text](#app-reg-dir1))

### Exercise 2

Use the `metrics.mean_squared_error` function to evaluate the loss
function used by `sklearn` when it fits the model for us.

Read the docstring to learn which the arguments that function takes.

In [ ]:
from sklearn import metrics

# your code here

([back to text](#app-reg-dir2))

### Exercise 3

Compare the mean squared error for the `lr_model` and the `sqft_lr_model`.

Which model has a better fit? Defend your choice.

([back to text](#app-reg-dir3))

### Exercise 4

Explore how you can improve the fit of the full model by adding additional
features created from the existing ones.

In [ ]:
# your code here

([back to text](#app-reg-dir4))

### Exercise 5

Experiment with how the size of the holdout dataset can impact a diagnosis
of overfitting.

Evaluate only the `LinearRegression` model on the full feature set and use
the `model_selection.train_test_split` function with various values for
`test_size`.

([back to text](#app-reg-dir5))

### Exercise 6

Read the documentation for `sklearn.tree.DecisionTreeRegressor` and
then experiment with adjusting some regularization parameters to see how they
affect the fitted tree.

In [ ]:
# plot trees when varying some regularization parameter(s)

([back to text](#app-reg-dir6))

### Exercise 7

Fit a regression tree to the housing price data and use graphviz
to visualize the decision graph.

In [ ]:
# your code here

([back to text](#app-reg-dir7))

### Exercise 8

Fit a random forest to the housing price data.

Compare the MSE on a testing set to that of lasso.

In [ ]:
# Fit random forest and compute MSE

Produce a bar chart of feature importances for predicting house
prices.

([back to text](#app-reg-dir8))

### Exercise 9

In the pseudocode below, fill in the blanks for the generic MLP.

Note that this is inside a markdown cell because the code is not valid
Python.

In [ ]:
ws = [w1, w2, ..., wend]
bs = [b1, b2, ..., bend]

def eval_mlp(X, ws, bs, f):
    """
    Evaluate MLP-given weights (ws), bias (bs) and an activation (f)

    Assumes that the same activation is applied to all hidden layers
    """
    N = len(ws) - 1

    out = X
    for i in range(N):
        out = f(__)  # replace the __

    # For this step remember python starts counting at 0!
    return out@__ + __  # replace the __

([back to text](#app-reg-dir9))

### Exercise 10

Scale all variables in `X` by subtracting their mean and dividing by the
standard deviation.

Verify that the transformed data has mean 0 and standard deviation 1.

In [ ]:
# your code here

([back to text](#app-reg-dir10))

### Exercise 11

Read the documentation for sklearn.neural_network.MLPRegressor and
use the full housing data to experiment with how adjusting layer depth, width, and other
regularization parameters affects prediction.

In [ ]:
# your code here